In [ ]:
import os
import wandb
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
wandb.login()

In [ ]:
# Get the dataset from UCI
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data -qq

# modified from https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py
# Import wandb
import wandb
import numpy as np
import xgboost as xgb

run = wandb.init(project="Lab1-visualize-models", name="xgboost")

# label need to be 0 to num_class -1
data = np.loadtxt('./dermatology.data', delimiter=',',
        converters={33: lambda x:int(x == '?'), 34: lambda x:int(x) - 1})
sz = data.shape

train = data[:int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7):, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6
wandb.config.update(param)

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5

# Add the wandb xgboost callback
bst = xgb.train(param, xg_train, num_round, watchlist, callbacks=[wandb.xgboost.WandbCallback()])
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

run.summary['Error Rate'] = error_rate

wandb.sklearn.plot_confusion_matrix(test_Y, pred, [0., 1., 2., 3., 4., 5.])

run.finish()